In [0]:
from __future__ import print_function
from builtins import range

import pandas as pd

#load
datatrain = pd.read_csv('LatentVars.csv')
# labeltrain = pd.read_csv('zinc.csv')

# xtrain = datatrain.values[1:,1:]
# ytrain = labeltrain.values[1:,2:3]
from sklearn.model_selection import train_test_split

train, test = train_test_split(datatrain, test_size=0.2)


In [0]:
xtrain = train.values[::,:196]
ytrain = train.values[::,198:199]


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

torch.manual_seed(1234)

#hyperparameters
hl = 1000
h2 = 100
h3 = 1000
lr = 0.0005
num_epoch = 251

#build model

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(196, hl)
        # self.fc1 = nn.BatchNorm1d(hl)
        # self.fc1 = nn.Dropout(0.2)
        # self.fc1 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hl, h3)
        self.fc3 = nn.BatchNorm1d(h3)
        self.fc3 = nn.Dropout(0.2)
        # self.fc4 = nn.Linear(h2, h3)
        self.fc2 = nn.Linear(h3, 1)

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc3(x))
        # x = F.relu(self.fc4(x))
        x = self.fc2(x)
        return x
net = Net()
net.cuda()



#choose optimizer and loss function
# criterion = nn.CrossEntropyLoss()

criterion = nn.MSELoss(size_average=None, reduce=None, reduction='mean')
# criterion = nn.KLDivLoss(size_average=None, reduce=None, reduction='mean')
# criterion = nn.L1Loss(size_average=None, reduce=None, reduction='mean')
# optimizer = torch.optim.SGD(net.parameters(), lr=lr)
optimizer = torch.optim.Adam(net.parameters())
# optimizer = torch.optim.SparseAdam(net.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08)
# optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

#train
for epoch in range(num_epoch):
      #  for batch_idx, df in datatrain.groupby(np.arange(len(datatrain)) // 10000):
            # xtrain = datatrain.values[1:10000,:196]
            X = Variable(torch.Tensor(xtrain).float()).cuda()
            # ytrain = datatrain.values[1:10000,196:197]
            Y = Variable(torch.Tensor(ytrain).float()).cuda()

            #feedforward - backprop
            optimizer.zero_grad()
            out = net(X)
            loss = criterion(out, Y)
            loss.backward()
            optimizer.step()

            if (epoch) % 50 == 0:
              print ('Epoch [%d/%d] Loss: %.4f' 
                            %(epoch+1, num_epoch, loss.item()))


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch [1/251] Loss: 10.3275
Epoch [51/251] Loss: 0.1448
Epoch [101/251] Loss: 0.0768
Epoch [151/251] Loss: 0.0598
Epoch [201/251] Loss: 0.0500
Epoch [251/251] Loss: 0.0440


In [0]:
# !kill -9 -1

In [0]:

xtest = test.values[::,:196]
ytest = test.values[::,198:199]

#get prediction
X = Variable(torch.Tensor(xtest).float()).cuda()
Y = Variable(torch.Tensor(ytest).float()).cuda()
out = net(X)
# _, predicted = torch.max(out.data, 1)
loss = criterion(out, Y)
#get accuration
# print('Accuracy of the network %d %%' % (100 * torch.sum(Y==predicted)))
print(loss)

tensor(0.0449, device='cuda:0', grad_fn=<MseLossBackward>)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [0]:
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,logP,qed,SAS
9112,0.037372,0.198021,0.874865,-1.597496,0.458180,2.086281,-0.200053,1.970732,-0.931601,0.101592,0.433848,-1.096730,1.622706,0.568950,-1.498358,0.110068,-0.009677,-0.392007,-1.305670,-0.802940,0.523539,1.569231,-0.035672,0.489622,1.522247,-0.519446,0.444397,1.452210,0.423653,-1.684398,-0.304499,2.206868,-0.224299,1.255183,-0.720282,-0.681987,-0.520052,-0.362997,-0.855875,0.366616,...,1.688287,-1.086755,-0.637041,1.579966,0.916889,1.115547,-0.387508,0.560892,-1.253853,1.944542,0.689061,1.850943,-2.418419,-2.370938,-0.063238,0.948149,1.095355,0.102300,0.670000,0.563123,-1.474750,-2.065071,-1.769692,1.132090,0.142503,-1.324836,-1.915201,-1.410888,-1.292959,-0.992574,1.607832,0.862677,-0.207658,1.596247,-1.572918,-0.914476,-1.008828,0.84540,0.857310,4.989014
18583,-0.862432,0.208225,0.314151,-1.335038,0.483515,1.032431,-0.555162,1.118352,-0.623638,-1.340408,0.923099,-0.308592,0.974843,-1.083682,-1.224195,0.581788,0.418890,-0.892407,-0.461648,-1.525855,0.348524,2.345390,0.271975,-0.261133,-1.344473,0.465094,-1.469653,-0.141198,0.843030,-0.028191,0.152255,0.620141,-0.047428,0.668780,1.902415,1.417804,-0.809457,-0.085432,-1.323008,-0.335828,...,0.609252,-0.667512,-0.601581,-0.071266,0.896589,-0.140552,0.177427,-0.194042,-0.139372,0.421297,2.636651,-1.168939,0.044389,-0.312635,1.365995,-1.203720,-1.443574,-0.302613,-0.278379,-2.283140,-1.519820,-0.941816,-1.388020,1.459666,1.084388,0.970528,-1.430401,-0.829667,-0.366574,-1.031208,0.239468,-0.101333,0.380973,-0.697925,0.557466,-1.297049,-2.074023,0.54004,0.898171,3.463921
108698,-0.065698,-0.161027,-1.515538,-0.311221,-0.472231,0.585864,-1.403468,-1.932610,2.062592,0.026209,-0.637858,-0.709213,0.280538,-2.530308,-0.251431,0.608879,-0.301247,-1.898938,-1.072156,0.837543,-0.771957,-0.632631,-0.222454,0.395289,0.242798,0.825790,-2.027379,1.130499,0.339140,0.924682,2.017657,0.204663,0.662162,-0.448383,0.077474,-0.994464,0.274470,1.435127,0.151788,-0.086421,...,-0.313160,-0.182022,-0.608676,0.000823,2.678732,-2.179658,-1.517312,0.387427,-0.394015,-1.023333,-0.254450,-2.014963,-1.224857,0.138713,-0.487836,0.101380,-0.073305,0.023632,-0.932386,0.803360,-1.676299,0.441657,0.100369,1.323463,0.696572,-0.891738,1.333587,-3.072316,0.772816,-1.615276,0.108612,-1.003512,-0.065904,-1.180000,-0.463603,0.238703,1.290721,2.39020,0.789792,3.633592
128753,0.098540,-1.897783,-0.347314,-0.476653,-0.543941,-0.047737,-0.096485,1.299027,-0.528106,0.950192,1.756898,0.173533,-0.885375,-0.004541,0.380086,1.924265,-1.054593,0.237868,-0.736690,-0.222551,0.726156,0.972636,0.653085,-0.251459,1.008332,0.009970,-1.368544,1.689787,1.307161,-0.939934,0.227904,0.350636,0.614880,1.588747,-0.249698,0.781937,0.390908,-1.451609,2.083805,-0.309661,...,0.665757,-1.300165,-0.163478,2.642613,1.327715,1.377415,-0.378450,-1.402916,0.566406,0.271459,0.557643,-0.218187,-1.459302,-1.066335,-0.334545,0.991848,0.480015,-0.824543,1.414498,1.293043,-1.071902,-1.367640,0.463775,1.179277,0.036265,-2.145636,-0.940488,-0.876094,0.928312,-2.227231,0.643340,0.315342,-0.247695,-0.295286,0.797470,1.251844,-0.198771,4.35390,0.751035,3.329700
162890,-1.610483,0.579621,0.207221,-0.124493,1.079294,-0.040467,-0.769885,0.141699,-1.593122,0.163371,-3.307173,-0.315600,-0.131612,0.438833,0.598314,0.287263,0.071619,-0.819679,-0.019283,-1.783236,-0.159686,-0.694175,1.543837,1.417636,-0.747798,-0.707203,-3.009867,0.104036,0.807672,-0.631519,0.134092,-1.252562,-2.232616,-0.949505,-1.337345,-1.453020,1.485858,1.430029,-0.875600,-1.810919,...,0.832327,1.815246,1.259578,-0.373269,-2.399624,-0.830430,0.576844,0.613384,-0.493887,-1.061322,-0.467263,-1.191724,-0.001121,0.238423,0.866476,1.767459,-0.093701,-0.552920,-0.560111,1.230536,-0.097804,-0.015084,-1.297668,-1.640953,1.362956,0.165559,0.